In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import numpy as np
import plotly.express as px
import dash_bootstrap_components as dbc
import geopandas as gpd
import mapboxgl

In [2]:
mapboxgl.access_token = 'pk.eyJ1IjoicjNuejNsIiwiYSI6ImNsdTNlZ2w2ODB6MjIyanFodDR6NXl2MnMifQ.kveM31v-0p75V41iiEt3bQ'

external_stylesheets = [dbc.themes.MINTY]
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "GROUP A - DATA101 Final Project"

# Loading datasets
main_df = pd.read_csv('https://raw.githubusercontent.com/R3NZ3L/DATA101-Final-Project/main/Datasets/CSV/dengue_cleaned.csv')
cdpp = pd.read_csv("https://raw.githubusercontent.com/R3NZ3L/DATA101-Final-Project/main/Datasets/CSV/cases_deaths_per_province.csv")
cdpr = pd.read_csv("https://raw.githubusercontent.com/R3NZ3L/DATA101-Final-Project/main/Datasets/CSV/cases_deaths_per_region.csv")
# regions = gpd.read_file("https://raw.githubusercontent.com/R3NZ3L/DATA101-Final-Project/main/Datasets/json/regions.json")
# provinces = gpd.read_file("datasets/json/provinces.json")
# regions_gdf = regions[["adm1_psgc", "geometry"]].merge(cdpr, on="adm1_psgc").set_index("adm1_psgc")


per_month = main_df[['region', 'month_num', 'month_name', 'cases', 'deaths']].groupby(['region', 'month_num', 'month_name'])[['cases', 'deaths']].sum().reset_index()
national = per_month.groupby(["month_num", "month_name"])[["cases", "deaths"]].sum().reset_index()
national["Region"] = "Philippines - National Aggregate"


In [3]:
import dash
from dash import html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px

per_month = main_df[['region', 'month_num', 'month_name', 'cases', 'deaths']].groupby(['region', 'month_num', 'month_name'])[['cases', 'deaths']].sum().reset_index()
national = per_month.groupby(["month_num", "month_name"])[["cases", "deaths"]].sum().reset_index()
national["Region"] = "Philippines - National Aggregate"

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Options for dropdown, including "All regions" to return back to all of the lines
dropdown_options = [{'label': 'All regions', 'value': 'all'}] + [{'label': region, 'value': region} for region in per_month['region'].unique()]



app.layout = html.Div([
    html.Div([
        dcc.Graph(
            figure={},
            id='bc_line'
        )
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Row([
                dbc.Col([       
                    html.H5("Line Depth:"),
                    dcc.RadioItems(
                        id='line_depth',
                        options=[
                            {'label': 'Regional', 'value': 'region'},
                            {'label': 'National', 'value': 'national'}
                        ],
                        value='region'  # Default value
                    )
                ]),
                dbc.Col([
                    html.H5("Variable:"),
                    dcc.RadioItems(
                        id='line_variable',
                        options=[
                            {'label': 'Cases', 'value': 'cases'},
                            {'label': 'Deaths', 'value': 'deaths'}
                        ],
                        value='cases'  # Default value
                    )
                ]),
                dbc.Col([
                    html.H5("Select Region:"),
                    dcc.Dropdown(
                        id='region_dropdown',
                        options=dropdown_options,
                        value='all', 
                         style={'width': '500px'}, # Default value
                        clearable=False  # Disable option to clear selection
                    )
                ])
            ])
        ], width=6)
    ])
])

@app.callback(
    Output("bc_line", "figure"),
    [Input("line_depth", "value"),
     Input("line_variable", "value"),
     Input("region_dropdown", "value")]
)
def update_bc_line(line_depth, line_variable, region_dropdown):
    if line_depth == "region":
        if region_dropdown == 'all':
            fig = px.line(per_month,
                          x="month_name",
                          y=line_variable,
                          color="region",
                          labels={
                              'month_name': 'Month',
                              'cases': 'Number of Cases',
                              'deaths': 'Number of Deaths',
                              'region': 'Region'
                          }
                         )
        else:
            filtered_data = per_month[per_month['region'] == region_dropdown]
            fig = px.line(filtered_data,
                          x="month_name",
                          y=line_variable,
                          color="region",
                          labels={
                              'month_name': 'Month',
                              'cases': 'Number of Cases',
                              'deaths': 'Number of Deaths',
                              'region': 'Region'
                          }
                         )
    elif line_depth == "national":
        fig = px.line(national,
                      x="month_name",
                      y=line_variable,
                      color="Region",
                      labels={
                          'month_name': 'Month',
                          'cases': 'Number of Cases',
                          'deaths': 'Number of Deaths',
                          'Region': 'Region'
                      }
                     )
        
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8052)

